# TBL data reduction

In [ ]:
import plopp as pp
from ess.reduce import time_of_flight
from ess import tbl
import ess.tbl.data  # noqa: F401
from ess.tbl.types import *

## Create and configure the workflow

In [ ]:
wf = tbl.TblWorkflow()

wf[Filename[SampleRun]] = tbl.data.tutorial_sample_data()

## nGEM detector data

In [ ]:
wf[NeXusDetectorName] = "ngem_detector"

wf.visualize(DetectorData[SampleRun], graph_attr={"rankdir": "LR"})

In [ ]:
ngem = wf.compute(DetectorData[SampleRun])
ngem

In [ ]:
ngem['dim_0', 0].hist().plot(aspect='equal') + ngem['dim_0', 1].hist().plot(aspect='equal')

### Time-of-flight

In [ ]:
wf.visualize(TofData[SampleRun], graph_attr={"rankdir": "LR"})

#### Cache the time-of-flight lookup table

In [ ]:
table = wf.compute(time_of_flight.TimeOfFlightLookupTable)
table.squeeze().plot()

In [ ]:
wf[TimeOfFlightLookupTable] = table

#### Results

In [ ]:
ngem_tofs = wf.compute(TofData[SampleRun])
ngem_wavs = wf.compute(WavelengthData[SampleRun])

ngem_tofs.bins.concat().hist(tof=100).plot() + ngem_wavs.bins.concat().hist(wavelength=100).plot()

## He3 detector data

In [ ]:
he3_wf = wf.copy()

he3 = {}
for bank in ('he3_detector_bank0', 'he3_detector_bank1'):
    he3_wf[NeXusDetectorName] = bank
    he3[bank] = he3_wf.compute(DetectorData[SampleRun])

In [ ]:
lims = {'vmin': 0, 'vmax': 4500}

he3['he3_detector_bank0'].hist().plot(title='he3_bank0', **lims) + he3['he3_detector_bank1'].hist().plot(title='he3_bank1', **lims)

### Time-of-flight

In [ ]:
he3_tofs = {}
he3_wavs = {}

for bank in ('he3_detector_bank0', 'he3_detector_bank1'):
    he3_wf[NeXusDetectorName] = bank
    he3_tofs[bank] = he3_wf.compute(TofData[SampleRun]).bins.concat().hist(tof=100)
    he3_wavs[bank] = he3_wf.compute(WavelengthData[SampleRun]).bins.concat().hist(wavelength=100)

pp.plot(he3_tofs) + pp.plot(he3_wavs)